## Investigate and rank distance between scottish destilleries and train stops

Data of Scottish whisky distilleries:
https://raw.githubusercontent.com/mhamilt/completely-smashed/main/completely-smashed.geojson

In [1]:
import pandas as pd
import geopandas as gpd

Read distillery data

In [2]:
try:
    distilleries = gpd.read_file('https://raw.githubusercontent.com/mhamilt/completely-smashed/main/completely-smashed.geojson')
except:
    # Fallback to local data
    distilleries = gpd.read_file('data/completely-smashed.geojson')

Prepare distillery data

In [3]:
# Manipulate data of Dalwhinnie Distillery, which is sligthly off. Actual position is at 56.939655,-4.2430025 (train station 56.934334,-4.246498)
distilleries.set_index(["Name"], inplace=True)
distilleries.at['Dalwhinnie Distillery', 'Lat'] = 56.939655
distilleries.at['Dalwhinnie Distillery', 'Lng'] = -4.2430025

# Delete one Loch Lomond distillery, this is a duplicate in the dataset
distilleries.drop("Loch Lomond Distillery (1)", inplace=True)

In [4]:
distilleries.geometry=gpd.points_from_xy(distilleries.Lng, distilleries.Lat)

In [5]:
distilleries.to_crs(epsg=27700, inplace=True)

Read UK train stops data

In [6]:
try:
    uk_stations = pd.read_csv('https://raw.githubusercontent.com/davwheat/uk-railway-stations/main/stations.csv', index_col=0)
except:
    uk_stations = pd.read_csv('data/stations.csv', index_col=0)

Prepare UK train stops data

In [7]:
uk_stations = gpd.GeoDataFrame(uk_stations, geometry=gpd.points_from_xy(uk_stations.long, uk_stations.lat))
uk_stations.set_crs('EPSG:4326', inplace=True)
uk_stations.to_crs(epsg=27700, inplace=True)

Find closest train stop to each distillery

In [8]:
distilleries_stops = distilleries.sjoin_nearest(uk_stations, distance_col="Distance in m")

Sort by distance & round

In [9]:
distilleries_stops = distilleries_stops.sort_values(by=["Distance in m"])
distilleries_stops["Distance in m"] = round(distilleries_stops["Distance in m"]).astype(int)

Filter columns and rename

In [10]:
distilleries_stops = distilleries_stops[["Owner", "Description", "Url", "Address", "index_right", "Distance in m", "geometry"]].rename(columns={"index_right": "Trainstop"})

Provide basic statistics

In [11]:
distilleries_stops.describe()

,Distance in m
count,136.000000
mean,19132.132353
std,24165.996763
min,383.000000
25%,2916.750000
50%,10781.000000
75%,23561.000000
max,123371.000000


In [12]:
distilleries_stops

,Owner,Description,Url,Address,Trainstop,Distance in m,geometry
Name,,,,,,,
Oban Distillery,Diageo plc,Highland Malt,https://www.malts.com/en-row/distilleries/oban/,,Oban,383,POINT (185922.846 730169.492)
The Borders Distillery,Three Stills Company Ltd,Lowland Malt,https://www.thebordersdistillery.com/,,Hawick,461,POINT (350163.815 614729.139)
Strathisla Distillery,Chivas Brothers Ltd,Speyside Malt,https://www.maltwhiskydistilleries.com/strathi...,,Keith,549,POINT (343035.051 851062.124)
Glen Keith Distillery,Chivas Brothers Ltd,Speyside Malt,,,Keith,615,POINT (342703.784 851072.092)
Dalwhinnie Distillery,Diageo plc,Highland Malt,https://www.malts.com/en-gb/distilleries/dalwh...,,Dalwhinnie,629,POINT (263630.447 785394.478)
...,...,...,...,...,...,...,...
Bowmore Distillery,Beam Suntory,Islay Malt,https://www.bowmore.com/,,Dunoon,88299,POINT (130891.450 659814.992)
Kilchoman Distillery,Kilchoman Distillery Co Ltd,Islay Malt,https://kilchomandistillery.com/,,Oban,89367,POINT (124494.045 664780.041)
Bruichladdich Distillery,Bruichladdich Distillery Co Ltd,Islay Malt,https://www.bruichladdich.com/,,Oban,89785,POINT (126703.348 662210.677)


In [13]:
# If you want to visit all distilleries:
total_distance=distilleries_stops["Distance in m"].sum() * 1e-3 # in km
print(f"The total distance is {total_distance:.2f} km")
print(f"Visiting all distillieries requires to run {total_distance/42.195:.2f} marathons")
print(f"The amount of whisky consumed assuming 3 x 35 ml per distillery is: {len(distilleries_stops.index)*3*0.035:.2f} litres")

The total distance is 2601.97 km
Visiting all distillieries requires to run 61.67 marathons
The amount of whisky consumed assuming 3 x 35 ml per distillery is: 14.28 litres


Save in various formats

In [14]:
# Change CRS to EPSG 4326
distilleries_stops.to_crs(epsg=4326, inplace = True)

# Save as GeoJson and use RFC7946 standard
distilleries_stops.to_file("output/distilleries_result.geojson", driver='GeoJSON', RFC7946="YES")

# Save as csv
distilleries_stops.to_csv("output/distilleries_result.csv")